# RapidHRV Tutorial

## Basic guide

In [1]:
import rapidhrv as rhv

We have included an example timeseries. This is a class containing a 5 minute segment of simulated PPG data and a specified sampling rate of 20Hz. This can be downloaded using the function below. When using your own data with RapidHRV however, the timeseries should be loaded into RapidHRV's Signal class with the specified sampling rate, e.g. Signal.from_csv('my_data.csv', sampling_rate=100).

In [2]:
# Download example signal

signal = rhv.get_example_data()

Using default preprocessing, a signal will then be upsampled to 1000Hz, high-pass filtered, and smoothed. The default analysis will use 'moderate' outlier rejection constraints. This can be altered via the outlier_detection_settings argument (accepting e.g. 'liberal', 'moderate', and 'conservative'; see our manuscript for further details).

In [3]:
# Preprocess and analyze data

preprocessed = rhv.preprocess(signal)
analyzed = rhv.analyze(preprocessed)

The data has now been fully analyzed and be plotted using rapidhrv's visualization tool. The GUI will allow you to select HR/HRV metrics and inspect the analyzed timeseries. Individual datapoints can then be selected to view the specified windows of extraction. To access the GUI, click on the URL link outputted in the console.

In [ ]:
# Visualize data

rhv.visualize(analyzed)

## Adjusting preprocessing parameters

The preprocessing steps implemented with RapidhRV will suffice for many uses. However, we have provided arguments which enable the user to alter preprocessing parameters. We list some examples below.

In [ ]:
# As RapidHRV upsamples to 1000Hz by default, this will need to be adjusted if your original sampling rate is 
# not a factor of 1000
preprocessed = rhv.preprocess(signal, resample_rate=500) 

# The high-pass filter is implemented with a cutoff of 0.5Hz by default, which can be changed with highpass_cutoff.
preprocessed = rhv.preprocess(signal, highpass_cutoff=0.05)

# To clean high frequency noise, RapidHRV imeplements Savitzky-Golay smoothing. The polynomial (default 3rd order)
# and window (default 100ms) can be changed. E.g. 4th order polynomial and a window width of 200ms:
preprocessed = rhv.preprocess(signal, sg_settings=(4,200))

# If you still want to apply low-pass filtering nonetheless (which is off by default), the user can specify a 
# frequency in the lowpass_cutoff argument to turn this on.
preprocessed = rhv.preprocess(signal, lowpass_cutoff=50)

## Adjusting analysis parameters

While we have strived to make RapidHRV a modality-general package, tweaks in analysis parameters may yield more sensitivty/accuracy, particularly the stringency of outlier rejection. To make this more user friendly we allow the use of semantically-labelled arguments. Below we list recommendations for dealing with different modalities.

In [ ]:
# For dealing with very clean ECG data, outlier rejection does not need to be too stringent. The user can thus 
# instruct RapidHRV to take a 'liberal' approach (default="moderate").  
analyzed = rhv.analyze(preprocessed, outlier_detection_settings="liberal")

# This should suit in most instances. However, for dealing atypical ECG signals, we have also enabled a k-means
# clustering algorithm to help delineate P, R, and T waves.
analyzed = rhv.analyze(preprocessed, outlier_detection_settings="liberal", ecg_prt_clustering=True)

# For dealing with data with moderate artifacts, RapidHRV's default settings should typically suffice. However,
# if the user wishes to have shorter/longer windows, with overlap between them, this can be specified. 
# For instance, to use 5 second windows with 1 second increments (4 second overlap), this can be specified by:
analyzed = rhv.analyze(preprocessed, window_width=5, window_overlap=4)

# When dealing with highly noisy data, such as PPG measures, more rigorous measures may be necessary. Here are
# recommended parameters for dealing with such data. These reduce the minimum amplitude for peak detection to 30%
# (from a default of 50%), increase window overlap, and apply more stringent outlier cleaning tools.
analyzed = rhv.analyze(preprocessed, outlier_detection_settings="conservative", window_overlap=9)